In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix
from tqdm import tqdm
from scipy.stats import binom

# Set seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# --- Configuration & Hyperparameters (Retained from previous steps) ---
NUM_CLASSES = 1
LEARNING_RATE = 0.001
NUM_EPOCHS = 100
BATCH_SIZE = 16
NUM_COMMITTEE_MEMBERS = 10
HIDDEN_ARCHITECTURES = [
    (64, 32, 0.5), (128, 64, 0.4), (32, 16, 0.6), (96, 48, 0.55), (160, 80, 0.3)
]
FEATURE_SETS_MAP = {
    'Top 50': {'X_train': 'X_train_top50.csv', 'y_train': 'y_train50.csv', 'X_test': 'X_test_top50.csv', 'y_test': 'y_test50.csv'},
    'Top 100': {'X_train': 'X_train_top100.csv', 'y_train': 'y_train100.csv', 'X_test': 'X_test_top100.csv', 'y_test': 'y_test100.csv'},
    'Top 250': {'X_train': 'X_train_top250.csv', 'y_train': 'y_train250.csv', 'X_test': 'X_test_top250.csv', 'y_test': 'y_test250.csv'},
    'All Genes': {'X_train': 'X_train_all.csv', 'y_train': 'y_train_all.csv', 'X_test': 'X_test_all.csv', 'y_test': 'y_test_all.csv'}
}


# --- 1. MLP Model Definition and Helper Functions ---

class SimpleMLP(nn.Module):
    # MLP definition remains the same
    def __init__(self, input_size, h1_size, h2_size, num_classes, dropout_rate):
        super(SimpleMLP, self).__init__()
        self.layer_1 = nn.Linear(input_size, h1_size)
        self.layer_2 = nn.Linear(h1_size, h2_size)
        self.layer_out = nn.Linear(h2_size, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        x = self.layer_1(x); x = self.relu(x); x = self.dropout(x)
        x = self.layer_2(x); x = self.relu(x)
        x = self.layer_out(x); x = self.sigmoid(x)
        return x

def train_model(model, train_loader, epochs, lr):
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model.train()
    for epoch in range(epochs):
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            loss = criterion(model(inputs), labels)
            loss.backward()
            optimizer.step()
    return model

def evaluate_raw_outputs(y_test_np, raw_outputs):
    """Calculates metrics based on raw outputs (used by both MLP and CNC)."""
    predictions = (raw_outputs >= 0.5).astype(int)
    y_true = y_test_np.flatten().astype(int)

    acc = accuracy_score(y_true, predictions)
    f1 = f1_score(y_true, predictions, average='binary')
    auc = roc_auc_score(y_true, raw_outputs)

    return {'Accuracy': acc, 'F1-Score': f1, 'ROC-AUC': auc, 'Predictions': predictions, 'TrueLabels': y_true}

# --- 2. Statistical Testing Function ---

def mcnemar_test(y_true, pred1, pred2):
    """
    Performs McNemar's test to compare two classifiers (Model 1 and Model 2).
    H0: The two classifiers have the same error rate.
    """
    # a: samples correctly predicted by M1 but incorrectly by M2
    # b: samples correctly predicted by M2 but incorrectly by M1

    y_true = y_true.flatten()

    # 1. Check where predictions differ
    correct1 = (y_true == pred1)
    correct2 = (y_true == pred2)

    # Calculate disagree counts (a and b)
    a = np.sum(np.logical_and(correct1, np.logical_not(correct2))) # M1 correct, M2 incorrect
    b = np.sum(np.logical_and(correct2, np.logical_not(correct1))) # M2 correct, M1 incorrect

    # McNemar's Test (using binomial exact test for small sample sizes)
    # The statistic follows B(n, 0.5) where n = a + b
    n = a + b

    # Calculate p-value: probability of observing a count of a or more extreme (2-sided test)
    # The test compares min(a, b) to n/2
    min_count = min(a, b)
    p_value = 2.0 * binom.cdf(min_count, n, 0.5)

    return {'a': a, 'b': b, 'n': n, 'p_value': p_value}

# --- 3. Model Training Functions (Modified to return predictions) ---

def load_data_for_comparison(X_train_path, y_train_path, X_test_path, y_test_path):
    """Loads all tensors and determines input size for a specific feature set."""
    try:
        X_train_df = pd.read_csv(X_train_path)
        y_train_df = pd.read_csv(y_train_path)
        X_test_df = pd.read_csv(X_test_path)
        y_test_df = pd.read_csv(y_test_path)
    except FileNotFoundError as e:
        print(f"Error: {e}. Skipping this feature set.")
        return None, None, None, None, None, None

    INPUT_SIZE = X_train_df.shape[1]
    y_train_col = y_train_df.columns[0]
    y_test_col = y_test_df.columns[0]

    X_train = torch.tensor(X_train_df.values, dtype=torch.float32)
    y_train = torch.tensor(y_train_df[y_train_col].values, dtype=torch.float32).unsqueeze(1)
    X_test = torch.tensor(X_test_df.values, dtype=torch.float32)
    y_test_np = y_test_df[y_test_col].values.reshape(-1, 1)

    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

    return INPUT_SIZE, X_train, y_train, X_test, y_test_np, train_loader

def train_and_evaluate_mlp(INPUT_SIZE, train_loader, X_test, y_test_np, feature_set_name):
    """Trains and evaluates a single baseline MLP."""
    h1, h2, dropout_rate = HIDDEN_ARCHITECTURES[0]
    mlp_model = SimpleMLP(INPUT_SIZE, h1, h2, NUM_CLASSES, dropout_rate)
    mlp_model = train_model(mlp_model, train_loader, NUM_EPOCHS, LEARNING_RATE)

    mlp_model.eval()
    with torch.no_grad():
        outputs = mlp_model(X_test).squeeze().numpy()

    metrics = evaluate_raw_outputs(y_test_np, outputs)
    metrics['Model Type'] = 'Single MLP'

    return metrics

def train_and_evaluate_cnc(INPUT_SIZE, X_train, y_train, X_test, y_test_np, feature_set_name):
    """Trains and evaluates the Diverse CNC."""
    num_train_samples = X_train.shape[0]
    num_architectures = len(HIDDEN_ARCHITECTURES)
    all_member_outputs = []

    # Training Diverse CNC
    for i in tqdm(range(NUM_COMMITTEE_MEMBERS), desc=f"   CNC {feature_set_name}"):
        bootstrap_indices = np.random.choice(range(num_train_samples), size=num_train_samples, replace=True)
        X_boot = X_train[bootstrap_indices]
        y_boot = y_train[bootstrap_indices]
        boot_dataset = TensorDataset(X_boot, y_boot)
        boot_train_loader = DataLoader(boot_dataset, batch_size=BATCH_SIZE, shuffle=True)

        h1, h2, dropout_rate = HIDDEN_ARCHITECTURES[i % num_architectures]
        member_model = SimpleMLP(INPUT_SIZE, h1, h2, NUM_CLASSES, dropout_rate)
        member_model = train_model(member_model, boot_train_loader, NUM_EPOCHS, LEARNING_RATE)

        member_model.eval()
        with torch.no_grad():
            member_outputs = member_model(X_test).squeeze().numpy()
            all_member_outputs.append(member_outputs)

    # Combine Predictions (Averaging)
    combined_outputs = np.stack(all_member_outputs, axis=0).mean(axis=0)

    # Evaluate
    metrics = evaluate_raw_outputs(y_test_np, combined_outputs)
    metrics['Model Type'] = 'Diverse CNC'

    return metrics

# --- 4. Final Reporting Functions ---

def report_results_and_confusion(results_df):
    """Prints the final summary and generates confusion matrices for ALL data points."""

    print("\n\n" + "="*70)
    print("FINAL MLP AND CNC DIMENSIONALITY REDUCTION IMPACT COMPARISON")
    print("="*70)
    print(results_df[['Feature Set', 'Model Type', 'Accuracy', 'F1-Score', 'ROC-AUC']].to_markdown(index=False))
    print("="*70)

    # Print Confusion Matrix for every result (for detailed reporting)
    for index, row in results_df.iterrows():
        y_true = row['TrueLabels']
        predictions = row['Predictions']
        cm = confusion_matrix(y_true, predictions)

        print("\n" + "-"*60)
        print(f"CONFUSION MATRIX: {row['Model Type']} on {row['Feature Set']}")
        print(f"Accuracy: {row['Accuracy']:.4f} | ROC-AUC: {row['ROC-AUC']:.4f}")
        print("-"*60)
        print("           Predicted 0 | Predicted 1")
        print(f"   True 0:  {cm[0, 0]:>10} | {cm[0, 1]:>10} (TN | FP)")
        print(f"   True 1:  {cm[1, 0]:>10} | {cm[1, 1]:>10} (FN | TP)")
        print("-" * 60)

# --- 5. Main Execution ---

if __name__ == "__main__":

    comparison_results = []
    best_results = {}

    for name, paths in FEATURE_SETS_MAP.items():

        INPUT_SIZE, X_train, y_train, X_test, y_test_np, train_loader = load_data_for_comparison(
            paths['X_train'], paths['y_train'], paths['X_test'], paths['y_test']
        )

        if INPUT_SIZE is None:
            continue

        print(f"\n{'='*70}\nStarting Analysis for: {name} (Input Size: {INPUT_SIZE})\n{'='*70}")

        # Run Single MLP
        mlp_metrics = train_and_evaluate_mlp(INPUT_SIZE, train_loader, X_test, y_test_np, name)
        mlp_metrics['Feature Set'] = name
        comparison_results.append(mlp_metrics)
        if mlp_metrics['ROC-AUC'] > best_results.get('MLP', {'ROC-AUC': 0})['ROC-AUC']:
            best_results['MLP'] = mlp_metrics

        # Run Diverse CNC
        cnc_metrics = train_and_evaluate_cnc(INPUT_SIZE, X_train, y_train, X_test, y_test_np, name)
        cnc_metrics['Feature Set'] = name
        comparison_results.append(cnc_metrics)
        if cnc_metrics['ROC-AUC'] > best_results.get('CNC', {'ROC-AUC': 0})['ROC-AUC']:
            best_results['CNC'] = cnc_metrics

    # Consolidate results and save
    results_df = pd.DataFrame(comparison_results)
    results_df.to_csv('mlp_cnc_dimensionality_comparison_full.csv', index=False)

    # Final Reporting (Metrics & Confusion Matrices)
    report_results_and_confusion(results_df)

    # --- 6. Statistical Validation on Best Models ---

    print("\n\n" + "="*70)
    print("STATISTICAL VALIDATION (McNEMAR'S TEST)")
    print("Comparing the BEST performing MLP vs. BEST performing CNC.")
    print(f"Best MLP (AUC {best_results['MLP']['ROC-AUC']:.4f}) on {best_results['MLP']['Feature Set']}")
    print(f"Best CNC (AUC {best_results['CNC']['ROC-AUC']:.4f}) on {best_results['CNC']['Feature Set']}")
    print("="*70)

    # Use the raw predictions from the best models found
    y_true = best_results['MLP']['TrueLabels'] # True labels are the same across all experiments (only test data changed)
    pred_mlp = best_results['MLP']['Predictions']
    pred_cnc = best_results['CNC']['Predictions']

    stat_result = mcnemar_test(y_true, pred_mlp, pred_cnc)

    print(f"McNEMAR's EXACT TEST RESULTS (n={stat_result['n']})")
    print("-" * 40)
    print(f"  MLP Correct, CNC Incorrect (a): {stat_result['a']}")
    print(f"  CNC Correct, MLP Incorrect (b): {stat_result['b']}")
    print("-" * 40)
    print(f"  P-value: {stat_result['p_value']:.4f}")
    print("\nInterpretation:")
    if stat_result['p_value'] < 0.05:
        print("The difference in performance between the two models is statistically significant (p < 0.05).")
    else:
        print("The difference in performance between the two models is NOT statistically significant (p > 0.05).")
    print("="*70)


Starting Analysis for: Top 50 (Input Size: 50)


   CNC Top 50: 100%|██████████| 10/10 [00:12<00:00,  1.21s/it]



Starting Analysis for: Top 100 (Input Size: 100)


   CNC Top 100: 100%|██████████| 10/10 [00:06<00:00,  1.45it/s]



Starting Analysis for: Top 250 (Input Size: 250)


   CNC Top 250: 100%|██████████| 10/10 [00:06<00:00,  1.53it/s]



Starting Analysis for: All Genes (Input Size: 7129)


   CNC All Genes: 100%|██████████| 10/10 [00:34<00:00,  3.48s/it]



FINAL MLP AND CNC DIMENSIONALITY REDUCTION IMPACT COMPARISON
| Feature Set   | Model Type   |   Accuracy |   F1-Score |   ROC-AUC |
|:--------------|:-------------|-----------:|-----------:|----------:|
| Top 50        | Single MLP   |   0.8      |   0.571429 |      0.8  |
| Top 50        | Diverse CNC  |   0.8      |   0.571429 |      0.86 |
| Top 100       | Single MLP   |   0.933333 |   0.888889 |      0.86 |
| Top 100       | Diverse CNC  |   0.933333 |   0.888889 |      0.88 |
| Top 250       | Single MLP   |   0.933333 |   0.888889 |      0.96 |
| Top 250       | Diverse CNC  |   0.933333 |   0.888889 |      0.96 |
| All Genes     | Single MLP   |   0.933333 |   0.888889 |      0.98 |
| All Genes     | Diverse CNC  |   0.933333 |   0.888889 |      0.96 |

------------------------------------------------------------
CONFUSION MATRIX: Single MLP on Top 50
Accuracy: 0.8000 | ROC-AUC: 0.8000
------------------------------------------------------------
           Predicted 0 | Predi